# Importações

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import io
import re
import os
import math
import time
import gdown
!pip install gdown
import shutil
from google.colab import files

# Preços Quitandaria

## Preços por KG

In [ ]:
# Lista de nomes e URLs dos produtos no Quitandaria
produtos_quitandaria = [
    ["Abacate", "https://maceio.quitandaria.com.br/produto/index/p/abacate-un"],
    ["Abobrinha", "https://maceio.quitandaria.com.br/produto/index/p/abobrinha-un"],
    ["Acerola", "https://maceio.quitandaria.com.br/produto/index/p/acerola-cumbuca-500g"],
    ["Alho Cabeça", "https://maceio.quitandaria.com.br/produto/index/p/alho-redinha-200g"],
    ["Ameixa", "https://maceio.quitandaria.com.br/produto/index/p/ameixa-fresca-nacional-un"],
    ["Banana Prata", "https://maceio.quitandaria.com.br/produto/index/p/banana-prata-6un"],
    ["Batata Doce", "https://maceio.quitandaria.com.br/produto/index/p/batata-doce-3un"],
    ["Batata Inglesa", "https://maceio.quitandaria.com.br/produto/index/p/batata-inglesa-un"],
    ["Cará", "https://maceio.quitandaria.com.br/produto/index/p/car-santom-1kg"],
    ["Castanha de Caju", "https://maceio.quitandaria.com.br/produto/index/p/castanha-de-caju-natural-100g-verdao"],
    ["Castanha do Pará", "https://maceio.quitandaria.com.br/produto/index/p/castanha-do-para-verdao-100g"],
    ["Cebola Branca", "https://maceio.quitandaria.com.br/produto/index/p/cebola-branca-2un"],
    ["Cebola Roxa", "https://maceio.quitandaria.com.br/produto/index/p/cebola-roxa-2un"],
    ["Cenoura", "https://maceio.quitandaria.com.br/produto/index/p/cenoura-un"],
    ["Goiaba Vermelha", "https://maceio.quitandaria.com.br/produto/index/p/goiaba-un"],
    ["Kiwi", "https://maceio.quitandaria.com.br/produto/index/p/kiwi-importado-un"],
    ["Laranja Lima", "https://maceio.quitandaria.com.br/produto/index/p/laranja-lima-un"],
    ["Laranja Pêra", "https://maceio.quitandaria.com.br/produto/index/p/laranja-pera-un"],
    ["Limão Taiti", "https://maceio.quitandaria.com.br/produto/index/p/limo-taiti-un"],
    ["Maçã Gala", "https://maceio.quitandaria.com.br/produto/index/p/ma-nacional-un"],
    ["Maçã Importada", "https://maceio.quitandaria.com.br/produto/index/p/ma-argentina-un"],
    ["Maçã Verde", "https://maceio.quitandaria.com.br/produto/index/p/ma-verde-un"],
    ["Macaxeira", "https://maceio.quitandaria.com.br/produto/index/p/macaxeira-3un"],
    ["Mamão Formosa", "https://maceio.quitandaria.com.br/produto/index/p/mamo-formosa-un"],
    ["Mamão Papaya", "https://maceio.quitandaria.com.br/produto/index/p/mamo-hava-gold-un"],
    ["Manga Espada", "https://maceio.quitandaria.com.br/produto/index/p/manga-espada-un"],
    ["Manga Rosa", "https://maceio.quitandaria.com.br/produto/index/p/manga-rosa-un"],
    ["Manga Tommy", "https://maceio.quitandaria.com.br/produto/index/p/manga-tomy-un"],
    ["Maracujá", "https://maceio.quitandaria.com.br/produto/index/p/maracuj-un"],
    ["Melancia", "https://maceio.quitandaria.com.br/produto/index/p/melancia-codama-un"],
    ["Melão Amarelo", "https://maceio.quitandaria.com.br/produto/index/p/melo-espanhol-redinha-un"],
    ["Morango", "https://maceio.quitandaria.com.br/produto/index/p/morango-crocante-peterfrut-250g"],
    ["Pepino", "https://maceio.quitandaria.com.br/produto/index/p/pepino-comum-un"],
    ["Pêra", "https://maceio.quitandaria.com.br/produto/index/p/pera-importada-danju-un"],
    ["Repolho Verde", "https://maceio.quitandaria.com.br/produto/index/p/repolho-verde-1-2un"],
    ["Tangerina Pokan", "https://maceio.quitandaria.com.br/produto/index/p/tangerina-nacional-1-kg"],
    ["Tomate Débora", "https://maceio.quitandaria.com.br/produto/index/p/tomate-2un"],
    ["Uva Verde", "https://maceio.quitandaria.com.br/produto/index/p/uva-thompson-verdo-500g"],
    ["Uva Vitória", "https://maceio.quitandaria.com.br/produto/index/p/uva-nacional-vitoria-verdo-500g"],
]

In [ ]:
# Função para extrair dados de um produto em uma URL
def scrape_product(url, nome_produto):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    product_box = soup.find('div', class_='produto-box')

    # Extrai o título do produto diretamente do HTML
    title_element = product_box.find('h1') if product_box else None
    title_text = title_element.text if title_element else nome_produto

    # Lógica de extração do preço
    price_element = product_box.find('p', class_='preco').find('span', class_='preco') if product_box else None
    price_text = price_element.text if price_element else None
    if price_text:
        price_text = price_text.replace('.', '').replace(',', '.')
        product_price = float(price_text) if price_text else None
    else:
        product_price = None
    extraction_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Lógica de extração da quantidade e unidade a partir do título
    match = re.search(r'(\d+(,\d+)?)\s*(KG|G|kg|g)', title_text, re.IGNORECASE)
    if match:
        quantity_text = match.group(1).replace(',', '.')
        quantity = float(quantity_text)
        unit = match.group(3).upper()
        if unit == 'G':
            price_per_kg = (product_price / quantity) * 1000 if product_price and quantity else None
        elif unit == 'KG':
            price_per_kg = product_price / quantity if product_price and quantity else None
    else:
        quantity = None
        unit = None
        price_per_kg = product_price

    if price_per_kg is not None:
        price_per_kg = round(price_per_kg, 2)
    price_per_kg_formatted = f'{price_per_kg:.2f}' if price_per_kg else None

    return {
        'name': nome_produto,
        'price': product_price,
        'quantity': quantity,
        'unit': unit,
        'price_per_kg': price_per_kg_formatted,
        'extraction_datetime': extraction_datetime,
        'store': 'Quitandaria'
    }

# Função principal
def main():
    all_products = []
    for nome_produto, url in produtos_quitandaria:
        product = scrape_product(url, nome_produto)
        all_products.append(product)
    df = pd.DataFrame(all_products)
    if not os.path.isfile('quitandaria_precos_kg.csv'):
        df.to_csv('quitandaria_precos_kg.csv', index=False)
    else:
        df.to_csv('quitandaria_precos_kg.csv', mode='a', header=False, index=False)

main()

# Carrega o CSV
df = pd.read_csv('quitandaria_precos_kg.csv')

# Substitui os pontos por vírgulas em todas as colunas
df = df.applymap(lambda x: str(x).replace('.', ','))

# Salva o DataFrame atualizado no CSV
df.to_csv('quitandaria_precos_kg.csv', index=False)

## Preços por Unidade

In [ ]:
# Lista combinada de nomes e links dos produtos
produtos_quitandaria = [
    ["Abacaxi", "https://maceio.quitandaria.com.br/produto/index/p/abacaxi-un"],
    ["Acelga", "https://maceio.quitandaria.com.br/produto/index/p/acelga-1-2-unidade"],
    ["Alface Crespa", "https://maceio.quitandaria.com.br/produto/index/p/alface-crespa-un"],
    ["Brócolis", "https://maceio.quitandaria.com.br/produto/index/p/brocolis-ninja-250g-verdao"],
    ["Cebolinha", "https://maceio.quitandaria.com.br/produto/index/p/cebolinha-un"],
    ["Coco Verde", "https://maceio.quitandaria.com.br/produto/index/p/coco-verde-un"],
    ["Coentro", "https://maceio.quitandaria.com.br/produto/index/p/coentro-un"],
    ["Couve Folha", "https://maceio.quitandaria.com.br/produto/index/p/couve-folha-un"],
]


In [ ]:
# Função para extrair dados de um produto em uma URL
def scrape_product(url, nome_produto):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    product_box = soup.find('div', class_='produto-box')

    # Extrai o título do produto diretamente do HTML
    title_element = product_box.find('h1') if product_box else None
    title_text = title_element.text if title_element else nome_produto

    # Lógica de extração do preço
    price_element = product_box.find('p', class_='preco').find('span', class_='preco') if product_box else None
    price_text = price_element.text if price_element else None
    if price_text:
        price_text = price_text.replace('.', '').replace(',', '.')
        product_price = float(price_text) if price_text else None
    else:
        product_price = None
    extraction_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    return {
        'name': nome_produto,  # Usar o nome fornecido na lista produtos_quitandaria
        'price': product_price,
        'extraction_datetime': extraction_datetime,
        'store': 'Quitandaria'
    }

# Função principal
def main():
    all_products = []
    for nome_produto, url in produtos_quitandaria:
        product = scrape_product(url, nome_produto)
        all_products.append(product)
    df = pd.DataFrame(all_products)
    if not os.path.isfile('quitandaria_precos_unidade.csv'):
        df.to_csv('quitandaria_precos_unidade.csv', index=False)
    else:
        df.to_csv('quitandaria_precos_unidade.csv', mode='a', header=False, index=False)

main()

# Carrega o CSV
df = pd.read_csv('quitandaria_precos_unidade.csv')

# Substitui os pontos por vírgulas em todas as colunas
df = df.applymap(lambda x: str(x).replace('.', ','))

# Salva o DataFrame atualizado no CSV
df.to_csv('quitandaria_precos_unidade.csv', index=False)

# Preços Gbarbosa

## Produtos por KG

In [ ]:
# Lista combinada de nomes e URLs dos produtos no Gbarbosa
produtos_gbarbosa = [
    ["Abóbora Leite", "https://www.gbarbosa.com.br/abobora-leite/p"],
    ["Abobrinha", "https://www.gbarbosa.com.br/abobrinha/p"],
    ["Alho Cabeça", "https://www.gbarbosa.com.br/alho-especial-granel/p"],
    ["Ameixa", "https://www.gbarbosa.com.br/ameixa-fresca-importada/p"],
    ["Banana Prata", "https://www.gbarbosa.com.br/banana-prata/p"],
    ["Batata Doce", "https://www.gbarbosa.com.br/batata-doce/p"],
    ["Batata Inglesa", "https://www.gbarbosa.com.br/batata-inglesa-lisa-bingi/p"],
    ["Castanha de Caju", "https://www.gbarbosa.com.br/castanha-de-caju-gbarbosa-natural-150g/p"],
    ["Castanha do Pará", "https://www.gbarbosa.com.br/castanha-do-para-gbarbosa-150g/p"],
    ["Cebola Branca", "https://www.gbarbosa.com.br/cebola-branca/p"],
    ["Cebola Roxa", "https://www.gbarbosa.com.br/cebola-roxa/p"],
    ["Cenoura", "https://www.gbarbosa.com.br/cenoura/p"],
    ["Goiaba Vermelha", "https://www.gbarbosa.com.br/goiaba-vermelha-pedaco/p"],
    ["Kiwi", "https://www.gbarbosa.com.br/kiwi-importado/p"],
    ["Laranja Lima", "https://www.gbarbosa.com.br/laranja-lima-da-persia/p"],
    ["Laranja Pêra", "https://www.gbarbosa.com.br/laranja-pera/p"],
    ["Limão Taiti", "https://www.gbarbosa.com.br/limao-taiti/p"],
    ["Maçã Gala", "https://www.gbarbosa.com.br/maca-nacional-vermelha/p"],
    ["Maçã Importada", "https://www.gbarbosa.com.br/maca-argentina/p"],
    ["Maçã Verde", "https://www.gbarbosa.com.br/maca-verde-importada/p"],
    ["Mamão Formosa", "https://www.gbarbosa.com.br/mamao-formosa-em-cubos/p"],
    ["Mamão Papaya", "https://www.gbarbosa.com.br/mamao-papaya/p"],
    ["Manga espada", "https://www.gbarbosa.com.br/manga-espada/p"],
    ["Manga rosa", "https://www.gbarbosa.com.br/manga-rosa/p"],
    ["Maracujá", "https://www.gbarbosa.com.br/maracuja/p"],
    ["Melancia", "https://www.gbarbosa.com.br/melancia-pedaco/p"],
    ["Melão Amarelo", "https://www.gbarbosa.com.br/melao-doce-redinha/p"],
    ["Pepino", "https://www.gbarbosa.com.br/pepino-comum/p"],
    ["Pêra", "https://www.gbarbosa.com.br/pera-importada/p"],
    ["Repolho Verde", "https://www.gbarbosa.com.br/repolho-verde/p"],
    ["Tangerina Pokan", "https://www.gbarbosa.com.br/tangerina-murcote-especial/p"],
    ["Tomate Débora", "https://www.gbarbosa.com.br/tomate/p"],
    ["Cará", "https://www.gbarbosa.com.br/inhame/p"],
    ["Tomate Italiano", "https://www.gbarbosa.com.br/tomate-italiano/p"],
    ["Uva Verde", "https://www.gbarbosa.com.br/uva-doce-mel-thompson-cumbuca-500g/p"],
    ["Morango", "https://www.gbarbosa.com.br/morango-nacional-bandeja-250g/p"],
    ["Uva Vitória", "https://www.gbarbosa.com.br/uva-fruta-nativa-vitoria-500g/p"]
]

In [ ]:
# Nome do arquivo CSV
csv_file = "gbarbosa_precos_kg.csv"

# Verifica se o arquivo CSV já existe
if os.path.isfile(csv_file):
    # Carrega o arquivo CSV existente em um DataFrame
    df = pd.read_csv(csv_file)
else:
    # Cria um DataFrame vazio se o arquivo não existe
    df = pd.DataFrame()

# Lista para armazenar os dados de cada página
data_list = []

for nome_produto, url in produtos_gbarbosa:
    # Faz a solicitação HTTP para a página
    response = requests.get(url)

    # Verifica se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")

        # Lógica para encontrar o nome do produto
        product_name = soup.find("span", class_="vtex-store-components-3-x-productBrand")
        product_name = product_name.get_text(strip=True) if product_name else "none"

        # Lógica para encontrar o preço
        price_elements = soup.select(".vtex-flex-layout-0-x-flexRow--price-details .gbarbosa-gbarbosa-components-0-x-currencyContainer")

        if len(price_elements) > 1:
            price_text = price_elements[1].text.strip()
            price_numeric = float(price_text.replace("R$", "").replace(",", ".").replace(",", "").replace(",", "").replace(",", ""))
        else:
            price_text = "none"
            price_numeric = "none"

        # Lógica para encontrar o elemento que contém a quantidade e a unidade
        asterisk = soup.find("small", class_="vtex-store-components-3-x-asterisk")

        if asterisk:
            # Extrai a quantidade e a unidade usando uma expressão regular
            match = re.search(r'(\d+(?:\.\d+)?)\s*([Kk]?[Gg])', asterisk.text)

            if match:
                quantity = float(match.group(1).replace(".", ""))
                unit = match.group(2).upper()
            else:
                quantity = "none"
                unit = "none"
        else:
            # Segunda tentativa: Extrai da URL
            url_match = re.search(r'(\d+)[Kk]?[Gg]', url)
            if url_match:
                quantity = int(url_match.group(1))
                unit = "G" if "g" in url.lower() else "KG"
            else:
                quantity = "none"
                unit = "none"

        # Lógica para calcular o preço por kg
        if unit != "none" and quantity != "none" and price_numeric != "none":
            if unit == "KG":
                price_per_kg = price_numeric
            elif unit == "G":
                price_per_kg = price_numeric / (quantity / 1000)
        else:
            # Se tanto a quantidade quanto a unidade são "none", define price_per_kg como o valor da coluna "price"
            price_per_kg = price_numeric if quantity == "none" and unit == "none" else "none"

        if price_per_kg != "none":
            # Arredonda para duas casas decimais usando math.floor
            price_per_kg = math.floor(price_per_kg * 100) / 100

        # Adiciona a data de extração
        extraction_date_time = datetime.now()

        # Formata a data e hora para incluir apenas os segundos
        extraction_date_time = extraction_date_time.strftime('%Y-%m-%d %H:%M:%S')

        # Cria um dicionário de dados
        data = {
            "name": nome_produto,
            "price": price_numeric,
            "quantity": quantity,
            "unit": unit,
            "price_per_kg": price_per_kg,
            "extraction_datetime": extraction_date_time,
            "store": "Gbarbosa"
        }

        data_list.append(data)
    else:
        print(f"Falha ao acessar a página {url}. Código de status:", response.status_code)

# Concatena os novos dados com o DataFrame existente
if data_list:
    new_data_df = pd.DataFrame(data_list)
    df = pd.concat([df, new_data_df], ignore_index=True)

# Salva o DataFrame em um arquivo CSV usando "," como separador decimal
df.to_csv(csv_file, index=False, decimal=',')

print(f"Dados salvos em '{csv_file}'")

Falha ao acessar a página https://www.gbarbosa.com.br/abobrinha/p. Código de status: 404
Falha ao acessar a página https://www.gbarbosa.com.br/castanha-do-para-gbarbosa-150g/p. Código de status: 404
Falha ao acessar a página https://www.gbarbosa.com.br/melao-doce-redinha/p. Código de status: 404
Falha ao acessar a página https://www.gbarbosa.com.br/tangerina-murcote-especial/p. Código de status: 404
Falha ao acessar a página https://www.gbarbosa.com.br/uva-fruta-nativa-vitoria-500g/p. Código de status: 404
Dados salvos em 'gbarbosa_precos_kg.csv'


## Preços por Unidade

In [ ]:
# Lista de nomes e URLs dos produtos no Gbarbosa
produtos_gbarbosa_unidade = [
    ["Abacaxi", "https://www.gbarbosa.com.br/abacaxi-perola-unid/p"],
    ["Acelga", "https://www.gbarbosa.com.br/acelga/p"],
    ["Brócolis", "https://www.gbarbosa.com.br/brocolis-chines-unid/p"],
    ["Coco Verde", "https://www.gbarbosa.com.br/coco-verde/p"],
    ["Coentro", "https://www.gbarbosa.com.br/coentro-unid/p"],
    ["Couve Folha", "https://www.gbarbosa.com.br/couve-folha-unid/p"]
]

In [ ]:
# Nome do arquivo CSV
csv_file = "gbarbosa_precos_unidade.csv"

# Verifica se o arquivo CSV já existe
if os.path.isfile(csv_file):
    # Carrega o arquivo CSV existente em um DataFrame
    df = pd.read_csv(csv_file)
else:
    # Cria um DataFrame vazio se o arquivo não existe
    df = pd.DataFrame()

# Lista para armazenar os dados de cada página
data_list = []

for nome_produto, url in produtos_gbarbosa_unidade:
    # Faz a solicitação HTTP para a página
    response = requests.get(url)

    # Verifica se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")

        # Lógica para encontrar o nome do produto
        product_name = soup.find("span", class_="vtex-store-components-3-x-productBrand")
        product_name = product_name.get_text(strip=True) if product_name else "none"

        # Lógica para encontrar o preço
        price_elements = soup.select(".vtex-flex-layout-0-x-flexRow--price-details .gbarbosa-gbarbosa-components-0-x-currencyContainer")

        if len(price_elements) > 1:
            price_text = price_elements[1].text.strip()
            price_numeric = float(price_text.replace("R$", "").replace(",", ".").replace(",", "").replace(",", "").replace(",", ""))
        else:
            price_text = "none"
            price_numeric = "none"

        # Adiciona a data de extração
        extraction_date_time = datetime.now()

        # Formata a data e hora para incluir apenas os segundos
        extraction_date_time = extraction_date_time.strftime('%Y-%m-%d %H:%M:%S')

        # Cria um dicionário de dados
        data = {
            "name": nome_produto,
            "price": price_numeric,
            "extraction_datetime": extraction_date_time,
            "store": "Gbarbosa"
        }

        data_list.append(data)
    else:
        print(f"Falha ao acessar a página {url}. Código de status:", response.status_code)

# Concatena os novos dados com o DataFrame existente
if data_list:
    new_data_df = pd.DataFrame(data_list)
    df = pd.concat([df, new_data_df], ignore_index=True)

# Salva o DataFrame em um arquivo CSV usando "," como separador decimal
df.to_csv(csv_file, index=False, decimal=',')

print(f"Dados salvos em '{csv_file}'")

Falha ao acessar a página https://www.gbarbosa.com.br/acelga/p. Código de status: 404
Falha ao acessar a página https://www.gbarbosa.com.br/couve-folha-unid/p. Código de status: 404
Dados salvos em 'gbarbosa_precos_unidade.csv'


# Preços Frutiver

## Preços por KG

In [ ]:
"""# Lista combinada de nomes e URLs dos produtos da Frutiver
produtos = [
    ["Abacate", "https://www.frutiver.com.br/produto/abacate-emb-600g/"],
    ["Abacaxi", "https://www.frutiver.com.br/produto/abacaxi-perola-extra-a-und-900g/"],
    ["Abóbora Leite", "https://www.frutiver.com.br/produto/abobora-leite-emb-11kg/"],
    ["Abobrinha", "https://www.frutiver.com.br/produto/abobrinha-italiana-emb-400g-media-1-a-2und/"],
    ["Acerola", "https://www.frutiver.com.br/produto/acerola-congelada-emb-1kg/"],
    ["Alho Cabeça", "https://www.frutiver.com.br/produto/alho-brco-imp-cabeca-50g/"],
    ["Ameixa", "https://www.frutiver.com.br/produto/ameixa-fresca-imp-emb-60g/"],
    ["Banana Prata", "https://www.frutiver.com.br/produto/banana-pacovan-extra-aaa-und-100g/"],
    ["Batata Doce", "https://www.frutiver.com.br/produto/batata-doce-rosada-emb-800g-media-2-a-3und/"],
    ["Batata Inglesa", "https://www.frutiver.com.br/produto/batata-inglesa-esp-emb-700g-media-5-a-6und/"],
    ["Brócolis", "https://www.frutiver.com.br/produto/brocolis-ninja-und-300g/"],
    ["Cará", "https://www.frutiver.com.br/produto/inhame-sao-tome-extra-a/"],
    ["Cebola Branca", "https://www.frutiver.com.br/produto/cebola-extra-aaa-emb-600g-media-3-a-4und/"],
    ["Cebola Roxa", "https://www.frutiver.com.br/produto/cebola-roxa-emb-600g-media-5-a-6und/"],
    ["Cebolinha", "https://www.frutiver.com.br/produto/cebolinha-und-50g/"],
    ["Cenoura", "https://www.frutiver.com.br/produto/cenoura-extra-aaa-emb-600g-media-2-a-3und/"],
    ["Coco Verde", "https://www.frutiver.com.br/produto/coco-verde-und/"],
    ["Coentro", "https://www.frutiver.com.br/produto/coentro-und-100g/"],
    ["Goiaba Vermelha", "https://www.frutiver.com.br/produto/goiaba-und-100g/"],
    ["Kiwi", "https://www.frutiver.com.br/produto/kiwi-imp-und-80g/"],
    ["Laranja Lima", "https://www.frutiver.com.br/produto/laranja-lima-extra-aaa-und-100g/"],
    ["Laranja Pêra", "https://www.frutiver.com.br/produto/laranja-pera-emb-25kg-media-11-a-12und/"],
    ["Limão Taiti", "https://www.frutiver.com.br/produto/limao-extra-aaa-und-60g/"],
    ["Maçã Gala", "https://www.frutiver.com.br/produto/maca-nacional-165-und/"],
    ["Maçã Verde", "https://www.frutiver.com.br/produto/maca-granny-smith-und-150g/"],
    ["Macaxeira", "https://www.frutiver.com.br/produto/macaxeira-extra-aaa/"],
    ["Mamão Formosa", "https://www.frutiver.com.br/produto/mamao-formosa-extra-aaa-12kg/"],
    ["Mamão Papaya", "https://www.frutiver.com.br/produto/mamao-havai-extra-aaa-350g/"],
    ["Manga Tommy", "https://www.frutiver.com.br/produto/manga-tomy-extra-aaa-und-300g/"],
    ["Maracujá", "https://www.frutiver.com.br/produto/maracuja-azedo-extra-aaa-emb-500g/"],
    ["Melancia", "https://www.frutiver.com.br/produto/melancia-extra-a-5kg/"],
    ["Melão Amarelo", "https://www.frutiver.com.br/produto/melao-espanhol-extra-aaa-und-13kg/"],
    ["Morango", "https://www.frutiver.com.br/produto/morango-fresco-bdj-300g/"],
    ["Pepino", "https://www.frutiver.com.br/produto/pepino-japones-emb-500g-media-2-a-3und/"],
    ["Pêra", "https://www.frutiver.com.br/produto/pera-danjou-und-133g/"],
    ["Repolho Verde", "https://www.frutiver.com.br/produto/repolho-verde-emb-12kg/"],
    ["Tomate Débora", "https://www.frutiver.com.br/produto/tomate-salada-l-vida-aaa-emb-500g-media-2-a-3und/"],
    ["Tomate Italiano", "https://www.frutiver.com.br/produto/tomate-p-molho-emb-500g-media-5-a-6und/"],
    ["Uva Verde", "https://www.frutiver.com.br/produto/uva-thompson-imp-s-s-vde-emb-500g/"],
    ["Uva Vitória", "https://www.frutiver.com.br/produto/uva-roxa-s-semente-emb-500g/"]
]"""

'# Lista combinada de nomes e URLs dos produtos da Frutiver\nprodutos = [\n    ["Abacate", "https://www.frutiver.com.br/produto/abacate-emb-600g/"],\n    ["Abacaxi", "https://www.frutiver.com.br/produto/abacaxi-perola-extra-a-und-900g/"],\n    ["Abóbora Leite", "https://www.frutiver.com.br/produto/abobora-leite-emb-11kg/"],\n    ["Abobrinha", "https://www.frutiver.com.br/produto/abobrinha-italiana-emb-400g-media-1-a-2und/"],\n    ["Acerola", "https://www.frutiver.com.br/produto/acerola-congelada-emb-1kg/"],\n    ["Alho Cabeça", "https://www.frutiver.com.br/produto/alho-brco-imp-cabeca-50g/"],\n    ["Ameixa", "https://www.frutiver.com.br/produto/ameixa-fresca-imp-emb-60g/"],\n    ["Banana Prata", "https://www.frutiver.com.br/produto/banana-pacovan-extra-aaa-und-100g/"],\n    ["Batata Doce", "https://www.frutiver.com.br/produto/batata-doce-rosada-emb-800g-media-2-a-3und/"],\n    ["Batata Inglesa", "https://www.frutiver.com.br/produto/batata-inglesa-esp-emb-700g-media-5-a-6und/"],\n    ["

In [ ]:
"""# Função para extrair dados de uma URL
def extrair_dados_frutiver(url, nome_produto):
    # Define um cabeçalho de usuário para a solicitação HTTP
    headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1"
}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')  # Analisa o HTML da página

    # Extrai o título da página
    title = soup.find('title').text

    # Extrai o preço atual do produto
    preco_atual = extrair_preco_atual(soup)

    # Extrai a quantidade e a unidade do título
    quantidade, unidade = extrair_quantidade_e_unidade(title)

    # Calcula o preço por quilo
    preco_por_quilo = calcular_preco_por_quilo(preco_atual, quantidade, unidade)

    # Obtém a data e hora da extração
    extraction_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Formata a quantidade com vírgula como separador de casas decimais
    quantidade_str = f"{quantidade:.2f}".replace('.', ',')

    # Verifica se o preço por quilo está disponível antes de formatá-lo como uma string
    if preco_por_quilo is not None:
        # Formata o preço por quilo com vírgula como separador de casas decimais
        preco_por_quilo_str = f"{preco_por_quilo:.2f}".replace('.', ',')
    else:
        preco_por_quilo_str = None

    # Retorna um dicionário com os dados do produto
    return {
        'name': nome_produto,
        'price': preco_atual,
        'quantity': quantidade_str,
        'unit': unidade,
        'price_per_kg': preco_por_quilo_str,
        'extraction_datetime': extraction_datetime,
        'store': 'Frutiver'
    }

# Função para calcular o preço por quilo
def calcular_preco_por_quilo(preco, quantidade, unidade):
    if preco is None:
        return None  # Retorna None para o preço por quilo se o preço não estiver disponível

    preco = float(preco.replace(',', '.'))  # Converte o preço em um número de ponto flutuante
    preco_por_quilo = None

    if quantidade:
        if unidade == 'G':
            preco_por_quilo = (preco / quantidade) * 1000  # Calcula o preço por quilo
        elif unidade == 'KG':
            preco_por_quilo = preco / quantidade  # Calcula o preço por quilo

    # Arredonda o valor para baixo com duas casas decimais
    preco_por_quilo = math.floor(preco_por_quilo * 100) / 100

    return preco_por_quilo

# Função para extrair o preço atual do produto do HTML
def extrair_preco_atual(soup):
    # Tenta encontrar o elemento com a classe "price"
    price_element = soup.find('p', class_='price')

    if price_element:
        # Procura dentro do elemento price por elementos span que contenham o preço
        price_span = price_element.find('span', class_='woocommerce-Price-amount')

        if price_span:
            # Obtém o texto do preço e remove o símbolo "R$"
            preco_atual = price_span.text.replace('R$', '').strip()

            if preco_atual.replace(',', '', 1).isdigit():  # Verifica se a string é um número
                return preco_atual

    return None  # Retorna None se o preço não for encontrado ou não for válido

# Função para extrair a quantidade e a unidade do título
def extrair_quantidade_e_unidade(title):
    # Procura por um número com casas decimais seguido da palavra "kg" ou "g" no título
    match = re.search(r'(\d+(?:,\d+)?)\s*([KGkg]{0,2})', title)
    if match:
        quantidade = float(match.group(1).replace(',', '.'))  # Obtém a quantidade como um número de ponto flutuante
        unidade = match.group(2).upper()  # Obtém a unidade em maiúsculas
        if not unidade:
            unidade = 'G'  # Define 'KG' como unidade padrão se não for especificada
    else:
        quantidade = 1  # Define como 1 se a unidade não for encontrada
        unidade = 'KG'  # Define 'KG' como unidade padrão
    return quantidade, unidade

# Função principal que coleta dados de várias URLs de produtos
def principal(produtos):
    todos_produtos = []
    for nome_produto, url in produtos:
        print(f"Processando {nome_produto} de {url}")
        try:
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            if response.status_code == 200:
                print("Resposta HTTP bem-sucedida.")
                produto = extrair_dados_frutiver(response.text, nome_produto)
                if produto:
                    print(f"Produto extraído: {produto}")
                    todos_produtos.append(produto)
                else:
                    print("Falha na extração do produto.")
            else:
                print(f"Falha na solicitação para {url} - Status Code: {response.status_code}")
        except requests.RequestException as e:
            print(f"Erro ao fazer a solicitação: {e}")

        #time.sleep(2)  # Atraso de 2 segundos entre cada solicitação

    if todos_produtos:
        df_local = pd.DataFrame(todos_produtos)
        print("DataFrame criado com sucesso.")
        print(df_local.head())
        df_local.to_csv('frutiver_precos_kg.csv', index=False)
        print("Arquivo CSV 'frutiver_precos_kg.csv' criado com sucesso.")
    else:
        print("Nenhum produto foi adicionado ao DataFrame.")

# Chama a função principal com a lista de produtos
principal(produtos)"""

'# Função para extrair dados de uma URL\ndef extrair_dados_frutiver(url, nome_produto):\n    # Define um cabeçalho de usuário para a solicitação HTTP\n    headers = {\n    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1"\n}\n    response = requests.get(url, headers=headers)\n    soup = BeautifulSoup(response.text, \'html.parser\')  # Analisa o HTML da página\n\n    # Extrai o título da página\n    title = soup.find(\'title\').text\n\n    # Extrai o preço atual do produto\n    preco_atual = extrair_preco_atual(soup)\n\n    # Extrai a quantidade e a unidade do título\n    quantidade, unidade = extrair_quantidade_e_unidade(title)\n\n    # Calcula o preço por quilo\n    preco_por_quilo = calcular_preco_por_quilo(preco_atual, quantidade, unidade)\n\n    # Obtém a data e hora da extração\n    extraction_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")\n\n    # Formata a quantidade 

## Preços por Unidade

In [ ]:
# Lista combinada de nomes e URLs dos produtos da Frutiver
produtos_unidade = [
    ["Abacaxi", "https://www.frutiver.com.br/produto/abacaxi-perola-extra-a-und-900g/"],
    ["Acelga", "https://www.frutiver.com.br/produto/acelga-emb-1kg/"],
    ["Alface Crespa", "https://www.frutiver.com.br/produto/alface-crespo-und-120g/"],
    ["Brócolis", "https://www.frutiver.com.br/produto/brocolis-ninja-und-300g/"],
    ["Coco Verde", "https://www.frutiver.com.br/produto/coco-verde-und/"],
    ["Coentro", "https://www.frutiver.com.br/produto/coentro-und-100g/"],
    ["Cebolinha", "https://www.frutiver.com.br/produto/cebolinha-und-50g/"],
    ["Couve Folha", "https://www.frutiver.com.br/produto/couve-folha-und-60g/"]
]

In [ ]:
# Função para extrair dados de uma URL
def extrair_dados_frutiver_unidade(url, nome_produto):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    preco_atual = extrair_preco_atual(soup)
    extraction_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    return {
        'name': nome_produto,
        'price': preco_atual,
        'extraction_datetime': extraction_datetime,
        'store': 'Frutiver'
    }

# Função para extrair o preço atual do produto do HTML
def extrair_preco_atual(soup):
    price_element = soup.find('p', class_='price')
    if price_element:
        price_span = price_element.find('span', class_='woocommerce-Price-amount')
        if price_span:
            preco_atual = price_span.text.replace('R$', '').strip()
            return preco_atual
    return None

# Função principal que coleta dados de várias URLs de produtos
def principal_unidade():
    todos_produtos = []
    for nome_produto, url in produtos_unidade:
        produto = extrair_dados_frutiver_unidade(url, nome_produto)
        if produto['name'] and produto['price']:
            todos_produtos.append(produto)

    if todos_produtos:
        df_local = pd.DataFrame(todos_produtos)
        df_local = df_local[['name', 'price', 'extraction_datetime', 'store']]
        df_local.to_csv('frutiver_precos_unidade.csv', index=False)

        # Exibe as primeiras linhas do DataFrame local
        print(df_local.head())

if __name__ == "__main__":
    principal_unidade()

            name  price  extraction_datetime     store
0        Abacaxi   4,49  2024-02-20 23:20:00  Frutiver
1         Acelga   6,99  2024-02-20 23:20:02  Frutiver
2  Alface Crespa   2,49  2024-02-20 23:20:04  Frutiver
3       Brócolis  16,99  2024-02-20 23:20:06  Frutiver
4     Coco Verde   4,49  2024-02-20 23:20:07  Frutiver


# Concatenando os arquivos

In [ ]:
# Carregue os 3 arquivos CSV em DataFrames individuais
# df1 = pd.read_csv('frutiver_precos_kg.csv')
df2 = pd.read_csv('gbarbosa_precos_kg.csv')
df3 = pd.read_csv('quitandaria_precos_kg.csv')

# Concatene os DataFrames verticalmente (um embaixo do outro)
concatenated_df = pd.concat([df2, df3], axis=0)

# Salve o DataFrame combinado como um novo arquivo CSV
concatenated_df.to_csv('precos_kg.csv', index=False)

# Verifique se o arquivo 'precos_kg.csv' foi criado com sucesso
import os
if os.path.exists('precos_kg.csv'):
    print("Arquivo 'precos_kg.csv' criado com sucesso.")
else:
    print("Algo deu errado ao criar o arquivo CSV combinado.")


Arquivo 'precos_kg.csv' criado com sucesso.


In [ ]:
# Carregue os 3 arquivos CSV em DataFrames individuais
df1 = pd.read_csv('frutiver_precos_unidade.csv')
df2 = pd.read_csv('gbarbosa_precos_unidade.csv')
df3 = pd.read_csv('quitandaria_precos_unidade.csv')

# Concatene os DataFrames verticalmente (um embaixo do outro)
concatenated_df = pd.concat([df2, df3], axis=0)

# Salve o DataFrame combinado como um novo arquivo CSV
concatenated_df.to_csv('precos_unidade.csv', index=False)

# Verifique se o arquivo 'precos_unidade.csv' foi criado com sucesso
import os
if os.path.exists('precos_unidade.csv'):
    print("Arquivo 'precos_unidade.csv' criado com sucesso.")
else:
    print("Algo deu errado ao criar o arquivo CSV combinado.")

Arquivo 'precos_unidade.csv' criado com sucesso.


#Ajustando Código do Produto

In [ ]:
# Função para substituir '.' por ','
def substituir_ponto_por_virgula(df):
    for coluna in df.columns:
        # Apenas se a coluna for do tipo object (string), substituir '.' por ','
        if df[coluna].dtype == 'object':
            df[coluna] = df[coluna].str.replace('.', ',', regex=False)
    return df

# Carregando os arquivos CSV
precos_kg_df = pd.read_csv('precos_kg.csv')
precos_unidade_df = pd.read_csv('precos_unidade.csv')

# Aplicando a substituição
precos_kg_df = substituir_ponto_por_virgula(precos_kg_df)
precos_unidade_df = substituir_ponto_por_virgula(precos_unidade_df)

# Sobrescrevendo os arquivos CSV existentes
precos_kg_df.to_csv('precos_kg.csv', index=False)
precos_unidade_df.to_csv('precos_unidade.csv', index=False)

print("Substituição de '.' por ',' concluída nos arquivos CSV.")

Substituição de '.' por ',' concluída nos arquivos CSV.


#Fazendo Download

In [ ]:
# Lista de nomes dos arquivos que você deseja baixar
arquivos_para_download = ['precos_kg.csv', 'precos_unidade.csv']

# Loop para fazer o download de cada arquivo
for arquivo in arquivos_para_download:
    files.download(arquivo)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>